In [104]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import jupyter_util as util
from importlib import reload
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/aries-acapy-plugin/controller_acapy/jupyter_util.py'>

## Enter Invite Url:

In [105]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNTlkOGM3N2MtZDRlNy00OWMyLTg3MDUtMjllZmUxYjc3NmE5IiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQxLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyI0SGtmang5SGNqS1JGMjg1c212RW5KUllrNVdKUTV6aGVWOVl0dUI5ZGdHVCJdLCAibGFiZWwiOiAiTWFpbiAoYWRtaW4pIn0=


In [106]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiOGExZWRhYjctOWZhNy00MzkwLWIyMTgtZmQ0N2I4NGFlNDZlIiwgInJlY2lwaWVudEtleXMiOiBbIjlhaUJVTmlKeXR4QmVRbzZvU0p3Qm9yVnpYVExFekVMdTVYRGRpdGgzNjJMIl0sICJsYWJlbCI6ICJDbGllbnQgKGFkbWluKSIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50Mi5sb2NhbGhvc3QifQ==


In [107]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': '8a1edab7-9fa7-4390-b218-fd47b84ae46e',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['9aiBUNiJytxBeQo6oSJwBorVzXTLEzELu5XDdith362L'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [108]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 2MYaxTDVxVAhiJS35mB6LYNU4gbMY3zjjYUuGfpryaStUzM9mSSxH8EmtMn5PUAJ88TVB9db1BKkihBy99cpwrf8

Public key: 
 BJTS4rZJ8gCiRMUdQfWicAuCbvgFo8okeoa6Yb9252zA


**Sending a connection request to acapy**

In [109]:
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [110]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [111]:
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

In [112]:
connection = util.connectAgents(agents[0], agents[1])
connection[1] = connection[1]['raw_repr']['connection_id']
connection[0] = connection[0]['connection_id']
print(connection[0], "\n", connection[1])

838516d7-dfa7-4d9c-859e-d4a0b9c030c3 
 5c435eae-a8e3-4c92-851e-1c81e1847cf1


In [113]:


# add = requests.post('http://localhost:8150/test-vs', json={
#   "connection_id": connection[0]
# })
# print(add.text)
# add = json.loads(add.text)

### FEATURE DISCOVERY

In [114]:
# pp.pprint(util.getFeatureDiscovery(agents[0]))

In [115]:
# consentSchema: function() {
#       return {
#         namespace: "consent",
#         DRI: "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s"
#       }
#     },

# consentAnswers: function() {
#       return [
#         "zQmNeoZGfo1chKACu9RLrVoG4afjLrGnAz1rhsfvGZwg9Z3",
#         "zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"
#       ]

In [116]:
add = requests.post('http://localhost:8151/verifiable-services/add', json={
  "label": "consent_test",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string"
  },
  "consent_schema": {
    "oca_schema_namespace": "consent",
    "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s",
    "data_url": "zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"
  }
})
add = json.loads(add.text)

newDid1 = util.createDid(agents[1])
newDid0 = util.createDid(agents[0])

try:
    util.registerDidOnLedger(agents[1], newDid1['result']['did'], newDid1['result']['verkey'])
except:
    pass
try:
    util.registerDidOnLedger(agents[0], newDid0['result']['did'], newDid0['result']['verkey'])
except:
    pass

util.setPublicDid(agents[1], newDid1['result']['did'])
util.setPublicDid(agents[0], newDid0['result']['did'])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '@id': '6c30160f-77cb-4566-847c-0980e0a37b0c',
 '~thread': {'thid': 'e9405436-64db-490f-bd1e-b5c47873696a'},
 'result': {'metadata': {'public': True, 'posted': True},
  'did': 'PaSe2EpSe8XwGAXmZFARcv',
  'verkey': 'DJh3ZzQqTjHyPFsdtw7eFJCueSc5PXmUE2RrbQ2rgrT1'}}

In [117]:
service_list = requests.get('http://localhost:8150/verifiable-services/DEBUGrequest/' + connection[0])
service_list = json.loads(service_list.text)
pp.pprint(service_list)

{'connection_id': '838516d7-dfa7-4d9c-859e-d4a0b9c030c3',
 'created_at': '2020-09-15 10:40:15.624910Z',
 'services': [{'consent_schema': {'data_url': 'zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G',
                                  'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s',
                                  'oca_schema_namespace': 'consent'},
               'label': 'consent_test',
               'service_id': 'f6171800-e112-43fc-84e3-01068ea4c69c',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}},
              {'consent_schema': {'data_url': 'zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G',
                                  'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s',
                                  'oca_schema_namespace': 'consent'},
               'label': 'consent_test',
               'service_id': '956a20d5-928c-4d5a-ad48-313d1c8f7eec',
            

In [118]:
apply = requests.post('http://localhost:8150/verifiable-services/apply', json={
  "connection_id": connection[0],
  "service_id": service_list['services'][0]['service_id'],
  "payload": "abc",
  "service": service_list['services'][0],
})
apply = json.loads(apply.text)
print(apply)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/verifiable-services/1.0/application', '@id': '6bad9120-c4f7-4ee3-aa29-792fb6ab9c9c', 'exchange_id': '656bc8c9-9f45-4dbe-9e51-547b31ff6774', 'credential_definition_id': 'PaSe2EpSe8XwGAXmZFARcv:3:CL:1066:consent_schema', 'data_dri': '38f31d5c233875b74152758a60a704e380be3efdd680c075b6955a0235ba7a4f', 'service_id': 'f6171800-e112-43fc-84e3-01068ea4c69c'}


In [119]:
issue_self = requests.post('http://localhost:8151/verifiable-services/get-issue-self', json={
    "exchange_id": apply['exchange_id'],
})
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

issue_self = requests.post('http://localhost:8151/verifiable-services/process-application', json={
    "issue_id": issue_self[0]['issue_id'],
    "decision": "accept",
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

[{'author': 'other',
  'connection_id': '5c435eae-a8e3-4c92-851e-1c81e1847cf1',
  'consent_schema': '{"oca_schema_namespace": "consent", "oca_schema_dri": '
                    '"fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s", '
                    '"data_url": '
                    '"zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"}',
  'created_at': '2020-09-15 10:40:39.015410Z',
  'exchange_id': '656bc8c9-9f45-4dbe-9e51-547b31ff6774',
  'issue_id': '0182c520e82fa605b64ed1d859c9a6f8038d30424b485998b5fa2555d1c84530',
  'label': 'consent_test',
  'payload': None,
  'service_id': 'f6171800-e112-43fc-84e3-01068ea4c69c',
  'service_schema': '{"oca_schema_namespace": "string", "oca_schema_dri": '
                    '"string"}',
  'state': 'pending',
  'updated_at': '2020-09-15 10:40:39.015410Z'}]
<Response [200]>
{'credential_exchange_record': {'auto_issue': True,
                                'auto_offer': False,
                                'auto_remove': False,
                    

In [36]:
payload = "some string yes dd"
payload_response = "response yay"
hash_id = hashlib.sha256(payload.encode("UTF-8")).hexdigest()
hash_id_response = hashlib.sha256(payload_response.encode("UTF-8")).hexdigest()
print(payload, hash_id)

some string yes dd b7c34035833ad394aaf721e431ae4a7f6359d66e1c69e8be9ec4c3e1d2e1ea1f


In [293]:
credentials = requests.get('http://localhost:8150/credentials')
credentials = json.loads(credentials.text) 
pp.pprint(credentials)

{'results': [{'attrs': {'consent_schema': '{"oca_schema_namespace": "string", '
                                          '"oca_schema_dri": "string", '
                                          '"data_url": "string"}',
                        'label': 'memxsassdsssdfdeds',
                        'service_schema': '{"oca_schema_namespace": "string", '
                                          '"oca_schema_dri": "string"}'},
              'cred_def_id': 'B6hy9V4FLjh6tQmknfEvTt:3:CL:273:Services',
              'cred_rev_id': None,
              'referent': 'e56cb0b5-f044-4176-b4d8-246c27ea0d00',
              'rev_reg_id': None,
              'schema_id': 'B6hy9V4FLjh6tQmknfEvTt:2:memxsassdsssdfdeds:1.0'},
             {'attrs': {'consent_schema': '{"oca_schema_namespace": "string", '
                                          '"oca_schema_dri": "string", '
                                          '"data_url": "string"}',
                        'label': 'memxsassdsssdfdeds',
         

I added returning hash id to add schema, I need to test if it works with both rejecting (not found schema) and accepting schema dound

### Add schema to storage

In [52]:
add = requests.post('http://localhost:8150/schema-storage/add', json={
  "payload": "test"
})
print(add.text)
add = json.loads(add.text)

{"hash_id": "9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08", "payload": "test"}


### SEND Initial Request

In [53]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": add['hash_id'],
  "connection_id": connection[1]
})

In [54]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": "ddd",
  "connection_id": connection[1]
})

In [55]:
print(send.text)

{"hash_id": "ddd", "connection_id": "a361d994-aaea-4f5c-9bf4-9644a774de2b", "record_id": "1e61b917-946b-4a56-8810-5b48081f79e6", "exchange_id": "c5f71769-b806-49fe-8861-15f49ffad9e3", "state": "pending"}


### GET Request By RECORD_ID

### GET Request List and filter by state and author

In [56]:
request_list = requests.post('http://localhost:8150/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("Agent1:", )
pp.pprint(request_list)
request_list = requests.post('http://localhost:8151/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("\nAgent2:")
pp.pprint(request_list)

Agent1:
[{'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:17.991960Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:17.991960Z'},
 {'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:16.748526Z',
  'exchange_id': 'f51e4a06-2214-4b9e-a28a-d4c336258f66',
  'payload': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'state': 'approved',
  'updated_at': '2020-07-16 09:09:16.748526Z'}]

Agent2:
[{'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:17.950785Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:18.029400Z'},
 {'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-

### GET RECORD LIST

In [57]:
getRequests = requests.post('http://localhost:8150/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent1: ")
pp.pprint(record_list)

getRequests = requests.post('http://localhost:8151/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent2: ")
pp.pprint(record_list)

Agent1: 
[{'author': 'self',
  'created_at': '2020-07-16 09:09:15.407083Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:15.407083Z'}]
Agent2: 
[{'author': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:16.808608Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:16.808608Z'}]


In [ ]:

# message = util.buildMessage(
#         'https://didcomm.org/issue-credential/1.0/propose-credential'
#     )
# testing = util.sendMessage(message, agents[0])
# pp.pprint(testing)

In [ ]:
# pp.pprint(util.sendSchema(agents[0], "schesssdddvd", "1.0", ["memesssaaaaaaa"]))

In [ ]:
# result = util.schemaGetList(agents[0])
# last_schema_id = result['results'][-1]['schema_id']
# print(result)

In [ ]:
# util.schemaGet(agents[0], last_schema_id)